## Tensorflow with example

Thanks to @edureka on youtube.com, I prepared the following notebook for a quick hands-on neural network model for a classification problem. 

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../data/Sonar.csv')

In [3]:
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V52,V53,V54,V55,V56,V57,V58,V59,V60,Class
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,1
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,1
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,1
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,1
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,1


**Preparing the sample and labels:** 

Note the .values applied to df. For later use in tensorflow, X should be an array not a pandas data frame. 

In [4]:
X = df.drop('Class', axis = 1).values
y = df['Class']

In [5]:
def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels, n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [6]:
y_o = one_hot_encode(y)

In [7]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y_o, test_size = 0.2, random_state = 0)

In [8]:
learning_rate = 0.3
training_epochs = 2000
cost_history = np.empty(shape=[1], dtype = float)
n_class = 2
model_path = "./dp_model/dp_solar_model"

We use a neural network with 6 layers (i.e. 4 hidden layers), with the following dimensions:

In [9]:
n_0 = X.shape[1] # this is equal to the number of features per sample
n_1 = 50 # size of hidden layer 1
n_2 = 50 # size of hidden layer 2
n_3 = 50 # size of hidden layer 3
n_4 = 50 # size of hidden layer 4
n_5 = len(np.unique(y)) # this is equal to the number of classes 

In [10]:
x_ = tf.placeholder(tf.float32, shape=[None, n_0], name = 'x_')
yreal_ = tf.placeholder(tf.float32, shape=[None, n_5], name = 'yreal_')

Overall, we have 6 layers: 0, 1, 2, 3, 4, 5. The first layer is denoted by $\boldsymbol{a}^{(0)}$ and the final layer is denoted by $a^{(L)}$ where $L = 5$. So, for each layer $l$, we have: $$\boldsymbol{a}^l = \mathrm{W}^l \boldsymbol{a}^{l - 1} + \boldsymbol{b}^l.$$

Even though for one sample it is convenient to write the linear map as above, with many samples we will rewrite the above forumlat as follows:
\begin{eqnarray}
\boldsymbol{a}^{(1)} = \boldsymbol{a}^{(l - 1)} \mathrm{W}^{(1)} + \boldsymbol{b}^{(1)}
\end{eqnarray}
where the dimenstion of $W$ is $n_0 \times n_1$.

Since $\boldsymbol{a}^{(0)}$ is indeed the input $(x)_{N \times n_0}$, where $N$ is the number of samples, the above formula reads as: 
\begin{eqnarray}
\boldsymbol{a}^{(1)} = \boldsymbol{x} \mathrm{W}^{(1)} + \boldsymbol{b}^{(1)}
\end{eqnarray}
Verify that the dimension of $\boldsymbol{a}^{(1)}$ is $N \times n_1$.

\begin{eqnarray}
\boldsymbol{z}^l = \boldsymbol{a}^{l - 1}  \mathrm{W}^l + \boldsymbol{b}^l
\end{eqnarray}


Therefore the dimensions of the weight matrices are as follows:

$\mathrm{W}^1$ is $n_0 \times n_1$

$\mathrm{W}^2$ is $n_1 \times n_2$

$\mathrm{W}^3$ is $n_2 \times n_3$

$\mathrm{W}^4$ is $n_3 \times n_4$

$\mathrm{W}^5$ is $n_4 \times n_5$

In [11]:
w = {
    '1' : tf.Variable(tf.truncated_normal([n_0, n_1])), 
    '2' : tf.Variable(tf.truncated_normal([n_1, n_2])),
    '3' : tf.Variable(tf.truncated_normal([n_2, n_3])),
    '4' : tf.Variable(tf.truncated_normal([n_3, n_4])),
    '5' : tf.Variable(tf.truncated_normal([n_4, n_5]))
}
b = {
    '1' : tf.Variable(tf.truncated_normal([n_1])),
    '2' : tf.Variable(tf.truncated_normal([n_2])),
    '3' : tf.Variable(tf.truncated_normal([n_3])),
    '4' : tf.Variable(tf.truncated_normal([n_4])),
    '5' : tf.Variable(tf.truncated_normal([n_5]))
}

The next step is to define the hypothesis function of the neural network. That is the formula for the output layer which is found by the sequence of $\boldsymbol{a}^l =  \boldsymbol{a}^{l - 1} \mathrm{W}^l + \boldsymbol{b}^l$

In [12]:
def multilayer_perception(x, w, b):
    z_1 = tf.add(tf.matmul(x, w['1']), b['1'])
    a_1 = tf.nn.sigmoid(z_1)
    
    z_2 = tf.add(tf.matmul(a_1, w['2']), b['2'])
    a_2 = tf.nn.sigmoid(z_2)
    
    z_3 = tf.add(tf.matmul(a_2, w['3']), b['3'])
    a_3 = tf.nn.sigmoid(z_3)
    
    z_4 = tf.add(tf.matmul(a_3, w['4']), b['4'])
    a_4 = tf.nn.relu(z_4)
    
    z_5 = tf.add(tf.matmul(a_4, w['5']), b['5'])
#     a_5 = tf.nn.softmax(z_5)
    
    return z_5

In [13]:
init = tf.global_variables_initializer()
ypred_ = multilayer_perception(x_, w, b)

In [14]:
cost_function_ = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=ypred_, labels=yreal_))

In [15]:
training_step_ = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function_)

In [16]:
sess = tf.Session()
sess.run(init)

In [17]:
Xtrain.shape

(166, 60)

In [18]:
cost_history = []
accuracy_history = []

In [19]:
correct_prediction_ = tf.equal(tf.argmax(yreal_, 1), tf.argmax(ypred_, 1))

In [20]:
accuracy_ = tf.reduce_mean(tf.cast(correct_prediction_, tf.float32))

In [21]:
for epoch in range(training_epochs):
    sess.run(training_step_, feed_dict={x_:Xtrain, yreal_:ytrain})
    cost = sess.run(cost_function_, feed_dict={x_:Xtrain, yreal_:ytrain})
    accuracy = sess.run(accuracy_, feed_dict={x_:Xtrain, yreal_:ytrain})
    cost_history = np.append(cost_history, cost)
    accuracy_history = np.append(accuracy_history, accuracy_)
    print("epoch: ", epoch, " - cost", cost, "- accuracy ", accuracy)

epoch:  0  - cost 80.47556 - accuracy  0.5421687
epoch:  1  - cost 10.079745 - accuracy  0.45783132
epoch:  2  - cost 6.368277 - accuracy  0.5421687
epoch:  3  - cost 1.3414011 - accuracy  0.45783132
epoch:  4  - cost 1.2389412 - accuracy  0.5421687
epoch:  5  - cost 0.7480049 - accuracy  0.5421687
epoch:  6  - cost 0.70358944 - accuracy  0.5421687
epoch:  7  - cost 0.68943435 - accuracy  0.5421687
epoch:  8  - cost 0.68654436 - accuracy  0.5421687
epoch:  9  - cost 0.68563634 - accuracy  0.5421687
epoch:  10  - cost 0.68489456 - accuracy  0.5421687
epoch:  11  - cost 0.684161 - accuracy  0.5421687
epoch:  12  - cost 0.68342334 - accuracy  0.5421687
epoch:  13  - cost 0.6826878 - accuracy  0.53614455
epoch:  14  - cost 0.6819691 - accuracy  0.5481928
epoch:  15  - cost 0.6812313 - accuracy  0.5481928
epoch:  16  - cost 0.6804772 - accuracy  0.5481928
epoch:  17  - cost 0.6796963 - accuracy  0.55421686
epoch:  18  - cost 0.67888504 - accuracy  0.55421686
epoch:  19  - cost 0.67806685 - 

epoch:  216  - cost 0.62519413 - accuracy  0.64457834
epoch:  217  - cost 0.49458846 - accuracy  0.79518074
epoch:  218  - cost 0.48558936 - accuracy  0.7409639
epoch:  219  - cost 0.48373395 - accuracy  0.78313255
epoch:  220  - cost 0.56201357 - accuracy  0.6987952
epoch:  221  - cost 0.46545404 - accuracy  0.813253
epoch:  222  - cost 0.4627977 - accuracy  0.75301206
epoch:  223  - cost 0.4844895 - accuracy  0.7710843
epoch:  224  - cost 0.61037165 - accuracy  0.6626506
epoch:  225  - cost 0.47218695 - accuracy  0.82530123
epoch:  226  - cost 0.47095734 - accuracy  0.7590361
epoch:  227  - cost 0.4987803 - accuracy  0.7710843
epoch:  228  - cost 0.6384404 - accuracy  0.6385542
epoch:  229  - cost 0.46093833 - accuracy  0.8313253
epoch:  230  - cost 0.43941453 - accuracy  0.7891566
epoch:  231  - cost 0.4288966 - accuracy  0.79518074
epoch:  232  - cost 0.419781 - accuracy  0.8012048
epoch:  233  - cost 0.416028 - accuracy  0.8012048
epoch:  234  - cost 0.43640587 - accuracy  0.77108

epoch:  445  - cost 0.43458462 - accuracy  0.7891566
epoch:  446  - cost 0.6606681 - accuracy  0.6987952
epoch:  447  - cost 0.53498757 - accuracy  0.6626506
epoch:  448  - cost 0.5246437 - accuracy  0.75301206
epoch:  449  - cost 0.28065547 - accuracy  0.8915663
epoch:  450  - cost 0.25632492 - accuracy  0.91566265
epoch:  451  - cost 0.24270141 - accuracy  0.92168677
epoch:  452  - cost 0.2327339 - accuracy  0.91566265
epoch:  453  - cost 0.2276722 - accuracy  0.92771083
epoch:  454  - cost 0.23154463 - accuracy  0.9096386
epoch:  455  - cost 0.25571233 - accuracy  0.91566265
epoch:  456  - cost 0.3733489 - accuracy  0.8012048
epoch:  457  - cost 0.28090703 - accuracy  0.88554215
epoch:  458  - cost 0.3962343 - accuracy  0.8012048
epoch:  459  - cost 0.3739174 - accuracy  0.8373494
epoch:  460  - cost 0.52464885 - accuracy  0.75301206
epoch:  461  - cost 0.39517137 - accuracy  0.82530123
epoch:  462  - cost 0.31984654 - accuracy  0.8433735
epoch:  463  - cost 0.35090035 - accuracy  0

epoch:  673  - cost 0.24803615 - accuracy  0.90361446
epoch:  674  - cost 0.22847447 - accuracy  0.91566265
epoch:  675  - cost 0.2194419 - accuracy  0.9096386
epoch:  676  - cost 0.25506157 - accuracy  0.90361446
epoch:  677  - cost 0.4981921 - accuracy  0.75301206
epoch:  678  - cost 0.25597417 - accuracy  0.9096386
epoch:  679  - cost 0.25572953 - accuracy  0.88554215
epoch:  680  - cost 0.21359877 - accuracy  0.92771083
epoch:  681  - cost 0.27752236 - accuracy  0.87349397
epoch:  682  - cost 0.24525638 - accuracy  0.90361446
epoch:  683  - cost 0.39465484 - accuracy  0.7891566
epoch:  684  - cost 0.24186596 - accuracy  0.90361446
epoch:  685  - cost 0.25790066 - accuracy  0.8915663
epoch:  686  - cost 0.23029546 - accuracy  0.91566265
epoch:  687  - cost 0.37017053 - accuracy  0.8072289
epoch:  688  - cost 0.23185234 - accuracy  0.92771083
epoch:  689  - cost 0.2509192 - accuracy  0.8915663
epoch:  690  - cost 0.23082504 - accuracy  0.9096386
epoch:  691  - cost 0.3829448 - accura

epoch:  888  - cost 2.1566036 - accuracy  0.5421687
epoch:  889  - cost 0.8273298 - accuracy  0.57831323
epoch:  890  - cost 0.6707782 - accuracy  0.57831323
epoch:  891  - cost 0.67248255 - accuracy  0.6385542
epoch:  892  - cost 0.6284997 - accuracy  0.57228917
epoch:  893  - cost 0.74264365 - accuracy  0.6325301
epoch:  894  - cost 0.4522787 - accuracy  0.813253
epoch:  895  - cost 0.44405687 - accuracy  0.7710843
epoch:  896  - cost 0.49611017 - accuracy  0.7590361
epoch:  897  - cost 0.4792989 - accuracy  0.71686745
epoch:  898  - cost 0.7262488 - accuracy  0.6385542
epoch:  899  - cost 0.4719453 - accuracy  0.79518074
epoch:  900  - cost 0.3676505 - accuracy  0.87349397
epoch:  901  - cost 0.3465415 - accuracy  0.85542166
epoch:  902  - cost 0.3974979 - accuracy  0.78313255
epoch:  903  - cost 0.6060715 - accuracy  0.7108434
epoch:  904  - cost 0.33333984 - accuracy  0.87349397
epoch:  905  - cost 0.35511363 - accuracy  0.82530123
epoch:  906  - cost 0.56418836 - accuracy  0.7228

epoch:  1110  - cost 0.038225945 - accuracy  0.9939759
epoch:  1111  - cost 0.0381807 - accuracy  0.9939759
epoch:  1112  - cost 0.037921812 - accuracy  0.9939759
epoch:  1113  - cost 0.038038097 - accuracy  0.9939759
epoch:  1114  - cost 0.03769829 - accuracy  0.9939759
epoch:  1115  - cost 0.03781478 - accuracy  0.9939759
epoch:  1116  - cost 0.037473157 - accuracy  0.9939759
epoch:  1117  - cost 0.03765077 - accuracy  0.9939759
epoch:  1118  - cost 0.03733953 - accuracy  0.9939759
epoch:  1119  - cost 0.037428673 - accuracy  0.9939759
epoch:  1120  - cost 0.037069198 - accuracy  0.9939759
epoch:  1121  - cost 0.037265725 - accuracy  0.9939759
epoch:  1122  - cost 0.036880374 - accuracy  0.9939759
epoch:  1123  - cost 0.03693202 - accuracy  0.9939759
epoch:  1124  - cost 0.036851376 - accuracy  0.9939759
epoch:  1125  - cost 0.03667847 - accuracy  0.9939759
epoch:  1126  - cost 0.036702577 - accuracy  0.9939759
epoch:  1127  - cost 0.036396887 - accuracy  0.9939759
epoch:  1128  - co

epoch:  1329  - cost 0.024608718 - accuracy  0.9939759
epoch:  1330  - cost 0.024079243 - accuracy  0.9939759
epoch:  1331  - cost 0.02438913 - accuracy  0.9939759
epoch:  1332  - cost 0.024103742 - accuracy  0.9939759
epoch:  1333  - cost 0.024344131 - accuracy  0.9939759
epoch:  1334  - cost 0.023973789 - accuracy  0.9939759
epoch:  1335  - cost 0.024283208 - accuracy  0.9939759
epoch:  1336  - cost 0.023865338 - accuracy  0.9939759
epoch:  1337  - cost 0.024203386 - accuracy  0.9939759
epoch:  1338  - cost 0.023790125 - accuracy  0.9939759
epoch:  1339  - cost 0.02373081 - accuracy  0.9939759
epoch:  1340  - cost 0.024008261 - accuracy  0.9939759
epoch:  1341  - cost 0.023747386 - accuracy  0.9939759
epoch:  1342  - cost 0.02397603 - accuracy  0.9939759
epoch:  1343  - cost 0.023462426 - accuracy  0.9939759
epoch:  1344  - cost 0.023490783 - accuracy  0.9939759
epoch:  1345  - cost 0.023632277 - accuracy  0.9939759
epoch:  1346  - cost 0.023725273 - accuracy  0.9939759
epoch:  1347 

epoch:  1545  - cost 0.017133106 - accuracy  0.9939759
epoch:  1546  - cost 0.017017089 - accuracy  0.9939759
epoch:  1547  - cost 0.017070899 - accuracy  0.9939759
epoch:  1548  - cost 0.016854692 - accuracy  0.9939759
epoch:  1549  - cost 0.016693823 - accuracy  0.9939759
epoch:  1550  - cost 0.016651679 - accuracy  0.9939759
epoch:  1551  - cost 0.01676839 - accuracy  0.9939759
epoch:  1552  - cost 0.016583208 - accuracy  0.9939759
epoch:  1553  - cost 0.016494025 - accuracy  0.9939759
epoch:  1554  - cost 0.017088676 - accuracy  0.9939759
epoch:  1555  - cost 0.016838277 - accuracy  0.9939759
epoch:  1556  - cost 0.016939605 - accuracy  0.9939759
epoch:  1557  - cost 0.016732834 - accuracy  0.9939759
epoch:  1558  - cost 0.016821533 - accuracy  0.9939759
epoch:  1559  - cost 0.016692426 - accuracy  0.9939759
epoch:  1560  - cost 0.016478986 - accuracy  0.9939759
epoch:  1561  - cost 0.016377835 - accuracy  0.9939759
epoch:  1562  - cost 0.016660951 - accuracy  0.9939759
epoch:  156

epoch:  1767  - cost 0.011929725 - accuracy  0.9939759
epoch:  1768  - cost 0.012402084 - accuracy  0.9939759
epoch:  1769  - cost 0.011993177 - accuracy  0.9939759
epoch:  1770  - cost 0.011806588 - accuracy  0.9939759
epoch:  1771  - cost 0.011729324 - accuracy  0.9939759
epoch:  1772  - cost 0.012245027 - accuracy  0.9939759
epoch:  1773  - cost 0.011906716 - accuracy  0.9939759
epoch:  1774  - cost 0.012130419 - accuracy  0.9939759
epoch:  1775  - cost 0.011982588 - accuracy  0.9939759
epoch:  1776  - cost 0.011781509 - accuracy  0.9939759
epoch:  1777  - cost 0.011685519 - accuracy  0.9939759
epoch:  1778  - cost 0.012027227 - accuracy  0.9939759
epoch:  1779  - cost 0.01181128 - accuracy  0.9939759
epoch:  1780  - cost 0.011999668 - accuracy  0.9939759
epoch:  1781  - cost 0.011821683 - accuracy  0.9939759
epoch:  1782  - cost 0.011647935 - accuracy  0.9939759
epoch:  1783  - cost 0.0115546575 - accuracy  0.9939759
epoch:  1784  - cost 0.011474769 - accuracy  0.9939759
epoch:  17

epoch:  1916  - cost 0.009199455 - accuracy  0.9939759
epoch:  1917  - cost 0.009273293 - accuracy  0.9939759
epoch:  1918  - cost 0.009933739 - accuracy  0.9939759
epoch:  1919  - cost 0.009406714 - accuracy  0.9939759
epoch:  1920  - cost 0.009682582 - accuracy  0.9939759
epoch:  1921  - cost 0.009383451 - accuracy  0.9939759
epoch:  1922  - cost 0.009245779 - accuracy  0.9939759
epoch:  1923  - cost 0.009137481 - accuracy  0.9939759
epoch:  1924  - cost 0.009119604 - accuracy  0.9939759
epoch:  1925  - cost 0.009808398 - accuracy  0.9939759
epoch:  1926  - cost 0.0092493715 - accuracy  0.9939759
epoch:  1927  - cost 0.009094135 - accuracy  0.9939759
epoch:  1928  - cost 0.009691601 - accuracy  0.9939759
epoch:  1929  - cost 0.009243434 - accuracy  0.9939759
epoch:  1930  - cost 0.009093436 - accuracy  0.9939759
epoch:  1931  - cost 0.009089096 - accuracy  0.9939759
epoch:  1932  - cost 0.009565675 - accuracy  0.9939759
epoch:  1933  - cost 0.009124648 - accuracy  0.9939759
epoch:  1

Now we evaluate the accuracy of the trained model on the test set:

In [22]:
sess.run(accuracy_, feed_dict={x_:Xtest, yreal_:ytest})

0.88095236

In [23]:
ypredLabel_ = tf.argmax(ypred_, 1)
ypredLabel = sess.run(ypredLabel_, feed_dict={x_:Xtest})

In [24]:
ytestLabel = np.argmax(ytest, 1)

In [25]:
pd.DataFrame(np.transpose([ypredLabel, ytestLabel]), columns=['Actual', 'Prediction'])

,Actual,Prediction
0,0,1
1,1,1
2,1,1
3,1,1
4,0,0
5,1,1
6,0,0
7,0,0
8,1,1
9,0,0
